In [ ]:
import numpy as np
import random
from EDAspy.optimization.multivariate import BOA
import logging
import matplotlib.pyplot as plt
logging.getLogger('pgmpy').setLevel(logging.WARNING)
import json

In [ ]:
def Rij(gene_length):
    return [[random.uniform(0.5, 1.0) for _ in range(gene_length)] 
            for _ in range(gene_length)]

In [ ]:
def fitness_function(genotype, rij):
    n = len(genotype) // 2
    i = sum(genotype[:n])
    j = sum(genotype[n:])
    R = rij[i - 1][j - 1]
    return R * (2 ** i + 2 ** j)

In [ ]:
def fitness_cost(genotype, rij):
    fitness_value = fitness_function(genotype, rij)
    return -fitness_value

In [ ]:
def setup_boa(gene_length, population_size):
    n_variables = 2 * gene_length
    possible_values = np.array([[0, 1] for _ in range(n_variables)])
    frequency = np.array([[0.5, 0.5] for _ in range(n_variables)])
    
    boa = BOA(
        size_gen=population_size,
        max_iter=10,
        dead_iter=2,
        n_variables=n_variables,
        alpha=0.5, 
        possible_values=possible_values,
        frequency=frequency,
        elite_factor=0.5,
        parallelize=False,
        disp=False
    )
    return boa

In [ ]:
def run_simulation(gene_length, amount_of_demes, population_per_deme, log=False):
    simulations_to_run = 30
    simulations = []
    for x in range(simulations_to_run):
        rij = Rij(gene_length)
        print("Simulation number:", simulations_to_run - x)
        generations = 0
        boa_demes = [setup_boa(gene_length, population_per_deme) for _ in range(amount_of_demes)]
        max_fitness_from_all_generations = 0
        best_possible_fitness = fitness_function([1] * (2 * gene_length), rij)
        no_more_diveristy = False
        while max_fitness_from_all_generations != best_possible_fitness and generations < 2000:
            generations += 1
            if log:
                print(f"Simulations Left: {simulations_to_run - x} || Generation:  {generations}")
            for boa_deme in boa_demes:
                if boa_deme.generation is None:
                    boa_deme.generation = boa_deme._initialize_generation()
                    boa_deme._check_generation(lambda sol: fitness_cost(sol, rij))
                    boa_deme.elite_temp = np.array([boa_deme.generation[0, :]])
                    boa_deme.evaluations_elite = np.array([boa_deme.evaluations.item(0)])
                else:
                    boa_deme._truncation()
                    boa_deme._update_pm()
                    boa_deme._new_generation()
                    boa_deme._check_generation(lambda sol: fitness_cost(sol, rij))
                deme_best_cost = np.min(boa_deme.evaluations)
                deme_best_fitness = -deme_best_cost
                max_fitness_from_all_generations = max(max_fitness_from_all_generations, deme_best_fitness)
            
            if log:
                print(max_fitness_from_all_generations, best_possible_fitness)
                
            if generations % 10 == 0:
                check_unique_positions = [0] * (2 * gene_length)
                for boa_deme in boa_demes:
                    for individual in boa_deme.generation:
                        for i in range(len(individual)):
                            if individual[i] == 1:
                                check_unique_positions[i] = 1
                check_unique_positions = set(check_unique_positions)
                if 0 in check_unique_positions:
                    print("some positions can't get 1, stopping simulation as we are stuck in optimum.")
                    no_more_diveristy = True
                    break
            
            check_unique = set()
            for boa_deme in boa_demes:
                for individual in boa_deme.generation:
                    check_unique.add(tuple(individual))
            if log:
                print(len(check_unique), check_unique)
            if len(check_unique) == 1 and max_fitness_from_all_generations != best_possible_fitness:
                no_more_diveristy = True
                print("No more diversity in the population, stopping simulation as we cannot reach optimum anymore.")
                break
            for deme_index, boa_deme in enumerate(boa_demes):
                deme_to_take_migrant_from = random.choice([i for i in range(amount_of_demes) if i != deme_index])
                migrant = random.choice(boa_demes[deme_to_take_migrant_from].generation)
                individual_to_replace = random.randint(0, len(boa_deme.generation) - 1)
                boa_deme.generation[individual_to_replace] = migrant
            
        if generations < 2000 and not no_more_diveristy:
            print("Found solution in", generations, "generations.", best_possible_fitness, "is the best possible fitness.", max_fitness_from_all_generations, "is the maximum fitness found.")
            simulations.append((generations, max_fitness_from_all_generations, best_possible_fitness))
    return simulations

In [ ]:
boa_length = [10, 20, 30, 40, 50, 60, 70, 80]

total_population = 400
amount_of_demes = 20
population_per_deme = total_population // amount_of_demes
boa_results = {}
for n in boa_length:
    print(f"Running simulation for gene length: {n}")
    results = run_simulation(n, amount_of_demes, population_per_deme, log=False)
    print(f"Finished simulation for gene length: {n}\n")
    if results != []:
        mean = np.mean(results)
        std_dev = np.std(results)
        boa_results[n] = {
            "mean": mean,
            "std": std_dev
        }
        print(f"Mutation_Only: Results for gene length {n}: {boa_results[n]}")
    else:
        print(f"Mutation_Only: Results for gene length {n}: COULDNT FIND SOLUTION!")
        
with open('boa_deme_migration_results.txt', 'w') as f:
    json.dump(boa_results, f, indent=4)

In [ ]:
boa_n_values = sorted(boa_results.keys())
boa_means = [boa_results[n]['mean'] for n in boa_results]
boa_stds = [boa_results[n]['std'] for n in boa_results]
    
plt.figure(figsize=(10, 6)) 
    
plt.plot(boa_n_values, boa_means, 
             marker='s', linestyle='-', color='blue', linewidth=2, 
             markersize=6, label='BOA with demes and migration')
    
for i, (x, y, std) in enumerate(zip(boa_n_values, boa_means, boa_stds)):
        plt.plot([x, x], [y-std, y+std], color='blue', linewidth=1)
        plt.plot([x-0.5, x+0.5], [y-std, y-std], color='blue', linewidth=1)
        plt.plot([x-0.5, x+0.5], [y+std, y+std], color='blue', linewidth=1)
    
plt.xlabel('n', fontsize=12)
plt.ylabel('generation to peak', fontsize=12)
plt.legend(loc='upper left', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(0, 90)
plt.ylim(0, 2000)
plt.xticks(range(0, 91, 10))
plt.yticks(range(0, 2001, 200))

plt.tight_layout()
plt.show()


In [ ]:
plt.semilogy(boa_n_values, boa_means, 
         marker='s', linestyle='-', color='blue', linewidth=2, 
         markersize=6, label='Boa with demes and migration')

for i, (x, y, std) in enumerate(zip(boa_n_values, boa_means, boa_stds)):
    plt.plot([x, x], [y-std, y+std], color='blue', linewidth=1)
    plt.plot([x-0.5, x+0.5], [y-std, y-std], color='blue', linewidth=1)
    plt.plot([x-0.5, x+0.5], [y+std, y+std], color='blue', linewidth=1)

plt.xlabel('n', fontsize=12)
plt.ylabel('generation to peak', fontsize=12)
plt.legend(loc='upper left', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(0, 90)
plt.ylim(1, 10000)
plt.xticks(range(0, 91, 10))

plt.tight_layout()
plt.show()